In [1]:
from re import compile
from eis1600.miu.methods import get_yml_and_miu_df

In [2]:
TOP_PATTERN = compile("T(?P<num_tokens>\d)(?P<category>[BDKMPR])")

In [3]:
categories = ["TOB", "TOD", "TOK", "TOM", "TOP", "TOR", "TOX"]
BIO = ["B", "I"]

labels = [bi + "-" + c for c in categories for bi in BIO] + ["O"]
print(labels)

['B-TOB', 'I-TOB', 'B-TOD', 'I-TOD', 'B-TOK', 'I-TOK', 'B-TOM', 'I-TOM', 'B-TOP', 'I-TOP', 'B-TOR', 'I-TOR', 'O']


In [4]:
label_dict = {}

for i, label in enumerate(labels):
  label_dict[label] = i

print(label_dict)

{'B-TOB': 0, 'I-TOB': 1, 'B-TOD': 2, 'I-TOD': 3, 'B-TOK': 4, 'I-TOK': 5, 'B-TOM': 6, 'I-TOM': 7, 'B-TOP': 8, 'I-TOP': 9, 'B-TOR': 10, 'I-TOR': 11, 'O': 12}


In [5]:
filepath = "../../Training_Data/gold_standard_topo/0748Dhahabi.TarikhIslam.MGR20180917-ara1.707162610706"
# filepath = "../../Training_Data/gold_standard_topo/0764Safadi.WafiBiWafayat.JK003609-ara1.843373946092"

with open(filepath + '.EIS1600', 'r', encoding='utf-8') as miu_file_object:
  yml, df = get_yml_and_miu_df(miu_file_object)

In [6]:
s_notna = df['TAGS_LISTS'].loc[df['TAGS_LISTS'].notna()].apply(lambda tag_list: ','.join(tag_list))
df_matches = s_notna.str.extract(TOP_PATTERN).dropna()
df_matches

,num_tokens,category
30,2,B
42,1,R
76,1,D


In [7]:
for index, row in df_matches.iterrows():
  processed_tokens = 0
  num_tokens = int(row['num_tokens'])
  while processed_tokens < num_tokens:
    print(processed_tokens, num_tokens, df.loc[index, 'TAGS_LISTS'])
    if processed_tokens == 0:
      df.loc[index, 'BIO'] = 'B-TO' + row['category']
    else:
      df.loc[index+processed_tokens, 'BIO'] = 'I-TO' + row['category']

    processed_tokens += 1

0 2 ['T2B']
1 2 ['T2B']
0 1 ['T1R']
0 1 ['T1D']


In [8]:
df[29:33]

,SECTIONS,TOKENS,TAGS_LISTS,BIO
29,None,وخمسمائة,None,NaN
30,None,بحصن,[T2B],B-TOB
31,None,كيفا,None,I-TOB
32,None,.,None,NaN


In [18]:
df["BIO"].loc[df["BIO"].isna()] = "O"